# Unity Catalog メタデータ抽出ノートブック

このノートブックはUnity Catalogから既存テーブルのメタデータを効率的に抽出し、TABLE_DDL_INFO と COLUMN_DDL_INFO の形式で整理します。

## 取得対象
- ◎: 完全自動取得可能項目
- ○: 条件付き自動取得可能項目  
- △: 推測可能項目
- ×: 手動管理必須項目（空値で初期化）

## 新機能 (v1.1)
- PK/FK制約情報の自動取得
- 並列処理による高速化
- 統合設定管理

## 1. 初期設定・パラメータ

In [ ]:
# ライブラリのインポート
import re
import json
from datetime import datetime, timezone
from concurrent.futures import ThreadPoolExecutor, as_completed
from pyspark.sql import functions as F
from pyspark.sql.types import *
from pyspark.sql import Row
from typing import Dict, Any, List, Tuple, Set

In [ ]:
# 対象のカタログを指定
catalog = "samples"

spark.sql(f"USE CATALOG {catalog}")

# カタログ配下のスキーマ一覧を取得
schemas_sql = f"""
SELECT 
    catalog_name, 
    schema_name, 
    schema_owner, 
    created, 
    last_altered 
FROM {catalog}.information_schema.schemata
WHERE 1=1
  AND schema_name <> 'information_schema'
ORDER BY catalog_name, schema_name
"""

schemas_df = spark.sql(schemas_sql)
display(schemas_df)

In [ ]:
# カタログ配下のスキーマ一覧を取得・確認
schema_list = [row["schema_name"] for row in schemas_df.select("schema_name").collect()]
schema_list

In [ ]:
# 統合設定（重複削除・一元管理）
CONFIG = {
    "target_catalog": catalog,
    "include_schemas": schema_list,
    "output_catalog": "ops",
    "output_path": "",
    "exclude_patterns": ['^__', '^event_log_'],
    "table_types": ['MANAGED', 'EXTERNAL', 'VIEW'],
    "retention_days": 180,
    "max_parallel_workers": 4,
    "describe_detail_timeout": 30
}

print(f"対象カタログ: {CONFIG['target_catalog']}")
print(f"対象スキーマ: {CONFIG['include_schemas']}")
print(f"実行時刻: {datetime.now()}")

## 2. 共通処理・ユーティリティ

In [ ]:
def build_filter_conditions(config):
    """共通フィルタ条件を生成（重複削除）"""
    schema_filter = "', '".join(config["include_schemas"])
    exclude_where = " AND ".join([f"table_name NOT RLIKE '{p}'" for p in config["exclude_patterns"]])
    table_types = "', '".join(config["table_types"])
    return schema_filter, exclude_where, table_types

def q(identifier: str) -> str:
    """Spark SQL用にバッククォートでエスケープ"""
    return f"`{identifier.replace('`', '``')}`"

def fqname(catalog: str, schema: str, table: str) -> str:
    """完全修飾名を生成"""
    return f"{q(catalog)}.{q(schema)}.{q(table)}"

## 3. テーブル基本情報取得

In [ ]:
# 共通フィルタ条件を使用
schema_filter, exclude_where, table_types = build_filter_conditions(CONFIG)

tables_sql = f"""
SELECT 
    table_catalog,
    table_schema,
    table_name,
    table_type,
    created,
    last_altered
FROM {CONFIG['target_catalog']}.information_schema.tables 
WHERE 1=1
  AND table_schema IN ('{schema_filter}')
  AND table_type IN ('{table_types}')
  AND {exclude_where}
ORDER BY table_catalog, table_schema, table_name
"""

tables_df = spark.sql(tables_sql)
tables_df.createOrReplaceTempView("base_tables")

table_count = tables_df.count()
print(f"取得テーブル数: {table_count}")

if table_count > 0:
    display(tables_df.limit(5))
else:
    print("対象テーブルはありません")

## 4. カラム基本情報の取得

In [ ]:
# 同じフィルタ条件を再利用
columns_sql = f"""
SELECT 
    table_catalog,
    table_schema,
    table_name,
    column_name,
    ordinal_position + 1 as ordinal_position,
    data_type,
    is_nullable,
    column_default,
    numeric_precision,
    numeric_scale
FROM {CONFIG['target_catalog']}.information_schema.columns 
WHERE 1=1 
  AND table_schema IN ('{schema_filter}')
  AND {exclude_where}
ORDER BY table_catalog, table_schema, table_name, ordinal_position
"""

columns_df = spark.sql(columns_sql)
columns_df.createOrReplaceTempView("base_columns")

column_count = columns_df.count()
print(f"取得カラム数: {column_count}")

if column_count > 0:
    display(columns_df.limit(5))
    display(columns_df.filter(F.column('data_type')== 'DECIMAL').limit(5))
else:
    print("対象カラムはありません")

## 4.5. PK/FK制約情報の取得

In [ ]:
# PRIMARY KEY制約情報を取得
pk_constraints_sql = f"""
SELECT 
    constraint_catalog,
    constraint_schema,
    table_name,
    constraint_name,
    constraint_type,
    column_names
FROM {CONFIG['target_catalog']}.information_schema.table_constraints 
WHERE 1=1
  AND constraint_type = 'PRIMARY KEY'
  AND constraint_schema IN ('{schema_filter}')
  AND {exclude_where.replace('table_name', 'table_name')}
ORDER BY constraint_catalog, constraint_schema, table_name
"""

try:
    pk_constraints_df = spark.sql(pk_constraints_sql)
    pk_constraints_df.createOrReplaceTempView("pk_constraints")
    pk_count = pk_constraints_df.count()
    print(f"取得PRIMARY KEY制約数: {pk_count}")
    if pk_count > 0:
        display(pk_constraints_df.limit(5))
except Exception as e:
    print(f"PRIMARY KEY制約取得エラー (スキップ): {e}")
    pk_constraints_df = spark.createDataFrame([], StructType([
        StructField("constraint_catalog", StringType()),
        StructField("constraint_schema", StringType()),
        StructField("table_name", StringType()),
        StructField("constraint_name", StringType()),
        StructField("constraint_type", StringType()),
        StructField("column_names", ArrayType(StringType()))
    ]))
    pk_constraints_df.createOrReplaceTempView("pk_constraints")

In [ ]:
# FOREIGN KEY制約情報を取得
fk_constraints_sql = f"""
SELECT 
    tc.constraint_catalog,
    tc.constraint_schema,
    tc.table_name,
    tc.constraint_name,
    tc.constraint_type,
    kcu.column_name,
    rc.referenced_table_catalog,
    rc.referenced_table_schema, 
    rc.referenced_table_name,
    rc.referenced_column_name
FROM {CONFIG['target_catalog']}.information_schema.table_constraints tc
JOIN {CONFIG['target_catalog']}.information_schema.key_column_usage kcu 
    ON tc.constraint_catalog = kcu.constraint_catalog 
    AND tc.constraint_schema = kcu.constraint_schema
    AND tc.constraint_name = kcu.constraint_name
JOIN {CONFIG['target_catalog']}.information_schema.referential_constraints rc
    ON tc.constraint_catalog = rc.constraint_catalog
    AND tc.constraint_schema = rc.constraint_schema  
    AND tc.constraint_name = rc.constraint_name
WHERE tc.constraint_type = 'FOREIGN KEY'
  AND tc.constraint_schema IN ('{schema_filter}')
  AND {exclude_where.replace('table_name', 'tc.table_name')}
ORDER BY tc.constraint_catalog, tc.constraint_schema, tc.table_name, kcu.ordinal_position
"""

try:
    fk_constraints_df = spark.sql(fk_constraints_sql)
    fk_constraints_df.createOrReplaceTempView("fk_constraints")
    fk_count = fk_constraints_df.count()
    print(f"取得FOREIGN KEY制約数: {fk_count}")
    if fk_count > 0:
        display(fk_constraints_df.limit(5))
except Exception as e:
    print(f"FOREIGN KEY制約取得エラー (スキップ): {e}")
    fk_constraints_df = spark.createDataFrame([], StructType([
        StructField("constraint_catalog", StringType()),
        StructField("constraint_schema", StringType()),
        StructField("table_name", StringType()),
        StructField("constraint_name", StringType()),
        StructField("constraint_type", StringType()),
        StructField("column_name", StringType()),
        StructField("referenced_table_catalog", StringType()),
        StructField("referenced_table_schema", StringType()),
        StructField("referenced_table_name", StringType()),
        StructField("referenced_column_name", StringType())
    ]))
    fk_constraints_df.createOrReplaceTempView("fk_constraints")

## 5. テーブル詳細(DESCRIBE DETAIL)情報を取得

In [ ]:
def describe_detail_one(catalog: str, schema: str, table: str) -> dict:
    """
    DESCRIBE DETAIL catalog.schema.table を1回実行して dict 化
    properties (MAP) は JSON 化もしておく
    """
    full_quoted = fqname(catalog, schema, table) 
    df = spark.sql(f"DESCRIBE DETAIL {full_quoted}")
    row = df.first()
    if row is None:
        return {
            "catalog_name": catalog,
            "schema_name": schema,
            "table_name": table,
            "full_table_name": f"{catalog}.{schema}.{table}",
            "error": "DESCRIBE DETAIL returned no rows"
        }
    d = row.asDict(recursive=True)

    # properties は MapType(string,string)（ない場合もある）。JSON化しておく
    props = d.get("properties")
    if isinstance(props, dict):
        d["properties_json"] = json.dumps(props, ensure_ascii=False, separators=(',', ':'))
    else:
        d["properties_json"] = None

    # フィールドを付与
    d.update({
        "catalog_name": catalog,
        "schema_name": schema,
        "table_name": table,
        "full_table_name": f"{catalog}.{schema}.{table}"
    })
    return d

def extract_clustering_info(table_details: Dict[str, Any]) -> Tuple[str, List[str]]:
    """
    DESCRIBE DETAIL の dict を前提に、クラスタリング方式と列を返す。
    """
    strategy = 'NONE'
    cols: List[str] = []

    features = table_details.get('tableFeatures', []) or []
    features_u = [str(f).upper() for f in features]

    props: Dict[str, Any] = table_details.get('properties', {}) or {}

    # Liquid clustering
    if 'LIQUID_CLUSTERING' in features_u:
        strategy = 'LIQUID'
        for key in ('delta.clusterBy', 'delta.liquidClustering.columns', 'delta.clusteredColumns'):
            v = props.get(key)
            if isinstance(v, str) and v.strip():
                cols = [c.strip() for c in v.split(',')]
                break

    # Z-Order
    if strategy == 'NONE':
        z_keys = [k for k in props.keys()
                  if k.lower().startswith('delta.zorder') or k.lower().endswith('zorderby')]
        if z_keys:
            strategy = 'ZORDER'
            for zk in z_keys:
                v = props.get(zk)
                if isinstance(v, str) and v.strip():
                    cols = [c.strip() for c in v.split(',')]
                    break

    return strategy, cols

def extract_partition_info(table_details: Dict[str, Any]) -> Tuple[str, List[str]]:
    """
    DESCRIBE DETAIL の partitionColumns（array<string>）をそのまま利用。
    """
    cols = table_details.get('partitionColumns', []) or []
    strategy = 'NONE' if not cols else 'BY_COLUMNS'
    return strategy, [str(c) for c in cols]

def _to_bool(s: Any) -> bool:
    return str(s).strip().lower() in ('true', '1', 'yes')

def get_delta_properties(table_details: Dict[str, Any]) -> Dict[str, Any]:
    """
    DESCRIBE DETAIL の properties/map と tableFeatures を用いて主な設定を抽出。
    """
    props: Dict[str, Any] = table_details.get('properties', {}) or {}
    features = table_details.get('tableFeatures', []) or []
    features_u = [str(f).upper() for f in features]

    out = {
        'auto_optimize_write':   _to_bool(props.get('delta.autoOptimize.optimizeWrite', 'false')),
        'auto_optimize_compact': _to_bool(props.get('delta.autoOptimize.autoCompact', 'false')),
        'cdf_enabled':           _to_bool(props.get('delta.enableChangeDataFeed', 'false')) or
                                 ('CHANGE_DATA_FEED' in features_u),
        'stats_column_limit':    int(props.get('delta.dataSkippingNumIndexedCols', 32) or 32),
        'stats_custom_columns':  None,
        'vacuum_retention_hours': 168,   # default
        'time_travel_retention_days': 30 # default
    }

    # dataSkipping のカスタム列
    scols = props.get('delta.dataSkippingStatsColumns')
    if isinstance(scols, str) and scols.strip():
        out['stats_custom_columns'] = [c.strip() for c in scols.split(',')]

    # vacuum の保持（例: "interval 168 hours" / "168 hours"）
    vstr = props.get('delta.deletedFileRetentionDuration')
    if isinstance(vstr, str) and vstr:
        m = re.search(r'(\d+)\s*hour', vstr, re.I)
        if m:
            out['vacuum_retention_hours'] = int(m.group(1))

    # time travel の保持（例: "interval 30 days" / "30 days"）
    lstr = props.get('delta.logRetentionDuration')
    if isinstance(lstr, str) and lstr:
        m = re.search(r'(\d+)\s*day', lstr, re.I)
        if m:
            out['time_travel_retention_days'] = int(m.group(1))

    return out

def list_target_tables():
    """base_tables からクオート付きで返す"""
    rows = spark.sql("""
        SELECT DISTINCT table_catalog, table_schema, table_name
        FROM base_tables
        ORDER BY table_catalog, table_schema, table_name
    """).collect()

    out = []
    for r in rows:
        out.append(Row(
            table_catalog=r.table_catalog,
            table_schema=r.table_schema,
            table_name=r.table_name,
            full_table_name_quoted=fqname(r.table_catalog, r.table_schema, r.table_name),
        ))
    return out

In [ ]:
def collect_details(table_rows, max_workers=None, limit=None):
    """並列版DESCRIBE DETAIL実行（推奨）"""
    if max_workers is None:
        max_workers = CONFIG.get("max_parallel_workers", 4)
    
    it = table_rows if limit is None else table_rows[:limit]
    print(f"並列実行開始: {len(it)}テーブル, {max_workers}並列")
    
    def process_table(r):
        try:
            return describe_detail_one(r.table_catalog, r.table_schema, r.table_name)
        except Exception as e:
            return {
                "catalog_name": r.table_catalog,
                "schema_name": r.table_schema,
                "table_name": r.table_name,
                "full_table_name": f"{r.table_catalog}.{r.table_schema}.{r.table_name}",
                "error": str(e)
            }
    
    results = []
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        future_to_table = {executor.submit(process_table, r): r for r in it}
        
        for future in as_completed(future_to_table):
            result = future.result()
            results.append(result)
            
            status = "OK" if 'error' not in result else "ERR"
            progress = f"({len(results)}/{len(it)})"
            print(f"{status} {progress}: {result['full_table_name']}")
    
    def sort_key(result):
        return (result.get('catalog_name', ''), 
                result.get('schema_name', ''), 
                result.get('table_name', ''))
    
    results.sort(key=sort_key)
    
    success_count = len([r for r in results if 'error' not in r])
    print(f"並列実行完了: 成功 {success_count}/{len(results)}")
    
    return results

In [ ]:
# 対象のリストの確認
tables = list_target_tables()

# 並列実行（推奨）- シリアル版より大幅に高速化
table_details = collect_details(tables, limit=None)

## 6. メタデータ統合処理

In [ ]:
def _human_bytes(n: Any) -> str | None:
    """バイト数に単位をつけて返す"""
    if n is None:
        return None
    try:
        n = int(n)
    except Exception:
        return None
    units = ["B","KB","MB","GB","TB","PB"]
    i = 0
    x = float(n)
    while x >= 1024 and i < len(units)-1:
        x /= 1024.0
        i += 1
    return f"{x:.2f} {units[i]}"

def build_constraint_maps() -> Tuple[Dict[Tuple[str, str, str], Set[str]], Dict[Tuple[str, str, str], List[Dict[str, str]]]]:
    """制約情報をインデックス化（PK列 / FK参照情報）"""
    pk_cols_map: Dict[Tuple[str, str, str], Set[str]] = {}
    fk_refs_map: Dict[Tuple[str, str, str], List[Dict[str, str]]] = {}
    
    # PRIMARY KEY情報の処理
    try:
        pk_rows = spark.sql("SELECT * FROM pk_constraints").collect()
        for row in pk_rows:
            table_key = (row.constraint_catalog, row.constraint_schema, row.table_name)
            col_names = row.column_names
            if isinstance(col_names, list):
                pk_cols_map[table_key] = set([c.lower() for c in col_names])
            elif isinstance(col_names, str):
                pk_cols_map[table_key] = set([c.strip().lower() for c in col_names.split(',')])
            else:
                pk_cols_map[table_key] = set()
    except Exception as e:
        print(f"PRIMARY KEY制約処理エラー: {e}")
    
    # FOREIGN KEY情報の処理
    try:
        fk_rows = spark.sql("SELECT * FROM fk_constraints").collect()
        for row in fk_rows:
            table_key = (row.constraint_catalog, row.constraint_schema, row.table_name)
            fk_info = {
                'column_name': row.column_name,
                'referenced_table': f"{row.referenced_table_catalog}.{row.referenced_table_schema}.{row.referenced_table_name}",
                'referenced_column': row.referenced_column_name,
                'constraint_name': row.constraint_name
            }
            if table_key not in fk_refs_map:
                fk_refs_map[table_key] = []
            fk_refs_map[table_key].append(fk_info)
    except Exception as e:
        print(f"FOREIGN KEY制約処理エラー: {e}")
    
    return pk_cols_map, fk_refs_map

def build_table_ddl_info(table_details: List[Dict[str, Any]]) -> List[Dict[str, Any]]:
    """TABLE_DDL_INFO構築のメイン処理"""
    idx: Dict[Tuple[str, str, str], Dict[str, Any]] = {}
    for d in table_details:
        k = (d.get('catalog_name'), d.get('schema_name'), d.get('table_name'))
        if all(k):
            idx[k] = d

    base_rows = spark.sql("""
        SELECT
            table_catalog, table_schema, table_name, table_type,
            created, last_altered
        FROM base_tables
        ORDER BY table_catalog, table_schema, table_name
    """).collect()

    out: List[Dict[str, Any]] = []
    for r in base_rows:
        key = (r.table_catalog, r.table_schema, r.table_name)
        detail = idx.get(key, {})
        
        cluster_strategy, cluster_cols = extract_clustering_info(detail)
        partition_strategy, partition_cols = extract_partition_info(detail)
        delta_props = get_delta_properties(detail) if detail else {
            'auto_optimize_write': False,
            'auto_optimize_compact': False,
            'cdf_enabled': False,
            'stats_column_limit': 32,
            'stats_custom_columns': None,
            'vacuum_retention_hours': 168,
            'time_travel_retention_days': 30,
        }
        
        rec = {
            'catalog_name': r.table_catalog,
            'schema_name':  r.table_schema,
            'table_name':   r.table_name,
            'table_type':   r.table_type,
            'storage_format':   detail.get('format', 'DELTA'),
            'storage_location': detail.get('location'),
            'external_location': detail.get('location') if r.table_type == 'EXTERNAL' else None,
            'partition_strategy':  partition_strategy,
            'partition_columns':   partition_cols,
            'clustering_strategy': cluster_strategy,
            'clustering_columns':  cluster_cols,
            'auto_optimize_write':   delta_props['auto_optimize_write'],
            'auto_optimize_compact': delta_props['auto_optimize_compact'],
            'vacuum_retention_hours': delta_props['vacuum_retention_hours'],
            'stats_column_limit':      delta_props['stats_column_limit'],
            'stats_custom_columns':    delta_props['stats_custom_columns'],
            'cdf_enabled':                 delta_props['cdf_enabled'],
            'time_travel_retention_days':  delta_props['time_travel_retention_days'],
            'num_files':      detail.get('numFiles'),
            'size_in_bytes':  detail.get('sizeInBytes'),
            'size_pretty':    _human_bytes(detail.get('sizeInBytes')),
            'table_features': [str(f) for f in (detail.get('tableFeatures') or [])],
            'table_id':     detail.get('id'),
            'created_at':   detail.get('createdAt', r.created),
            'last_altered': detail.get('lastModified', r.last_altered),
            'extracted_at': datetime.now(timezone.utc).isoformat(),
            'extraction_method': 'detail_python',
        }
        
        if 'error' in detail:
            rec['detail_error'] = detail['error']
            
        out.append(rec)

    return out

def build_column_ddl_info(table_details: List[Dict[str, Any]]) -> List[Dict[str, Any]]:
    """COLUMN_DDL_INFO構築のメイン処理"""
    # パーティション・クラスタ列のマップ作成
    part_cols_map: Dict[Tuple[str, str, str], Set[str]] = {}
    clus_cols_map: Dict[Tuple[str, str, str], Set[str]] = {}

    for d in table_details:
        key = (d.get('catalog_name'), d.get('schema_name'), d.get('table_name'))
        if not all(key):
            continue
        _, part_cols = extract_partition_info(d)
        part_cols_map[key] = set([c.lower() for c in part_cols])
        _, clus_cols = extract_clustering_info(d)
        clus_cols_map[key] = set([c.lower() for c in clus_cols])

    # PK/FK制約情報の取得
    pk_cols_map, fk_refs_map = build_constraint_maps()

    # 列のベース情報を取得
    rows = spark.sql("""
        SELECT *
        FROM base_columns
        ORDER BY table_catalog, table_schema, table_name, ordinal_position
    """).collect()

    out: List[Dict[str, Any]] = []
    now_utc = datetime.now(timezone.utc).isoformat()

    for r in rows:
        key = (r.table_catalog, r.table_schema, r.table_name)
        pset = part_cols_map.get(key, set())
        cset = clus_cols_map.get(key, set())
        pkset = pk_cols_map.get(key, set())
        fk_refs = fk_refs_map.get(key, [])
        
        col_name_lower = r.column_name.lower()
        
        # FK参照情報をこのカラムでフィルタ
        col_fk_refs = [fk for fk in fk_refs if fk['column_name'].lower() == col_name_lower]
        fk_reference = None
        if col_fk_refs:
            fk_ref = col_fk_refs[0]
            fk_reference = f"{fk_ref['referenced_table']}.{fk_ref['referenced_column']}"

        rec = {
            "catalog_name":  r.table_catalog,
            "schema_name":   r.table_schema,
            "table_name":    r.table_name,
            "column_name":   r.column_name,
            "ordinal_position": int(r.ordinal_position),
            "data_type":          r.data_type,
            "numeric_precision":  r.numeric_precision,
            "numeric_scale":      r.numeric_scale,
            "is_nullable":     (str(r.is_nullable).upper() == "YES"),
            "default_value":   r.column_default,
            "column_comment":  getattr(r, "comment", None) or getattr(r, "column_comment", None),
            "is_partition_column": col_name_lower in pset,
            "is_clustering_column": col_name_lower in cset,
            "is_primary_key": col_name_lower in pkset,
            "foreign_key_reference": fk_reference,
            "extracted_at": now_utc,
            "extraction_method": "detail_python",
        }
        
        out.append(rec)

    return out

In [ ]:
# メタデータ構築を実行
table_ddl_data = build_table_ddl_info(table_details)
column_ddl_data = build_column_ddl_info(table_details)

print(f"TABLE_DDL_INFO構築完了: {len(table_ddl_data)}件")
print(f"COLUMN_DDL_INFO構築完了: {len(column_ddl_data)}件")

## 7. DataFrame作成と保存

In [ ]:
# TABLE_DDL_INFO DataFrame作成
if table_ddl_data:
    table_schema = StructType([
        StructField("catalog_name", StringType(), False),
        StructField("schema_name",  StringType(), False),
        StructField("table_name",   StringType(), False),
        StructField("table_type",   StringType(), False),
        StructField("storage_format",   StringType(), True),
        StructField("storage_location", StringType(), True),
        StructField("external_location", StringType(), True),
        StructField("partition_strategy", StringType(), False),
        StructField("partition_columns",  ArrayType(StringType()), True),
        StructField("clustering_strategy", StringType(), False),
        StructField("clustering_columns",  ArrayType(StringType()), True),
        StructField("auto_optimize_write",   BooleanType(), False),
        StructField("auto_optimize_compact", BooleanType(), False),
        StructField("vacuum_retention_hours", IntegerType(), True),
        StructField("stats_column_limit",     IntegerType(), True),
        StructField("stats_custom_columns",   ArrayType(StringType()), True),
        StructField("cdf_enabled",                 BooleanType(), False),
        StructField("time_travel_retention_days",  IntegerType(), True),
        StructField("num_files",     LongType(), True),
        StructField("size_in_bytes", LongType(), True),
        StructField("size_pretty",   StringType(), True),
        StructField("table_features", ArrayType(StringType()), True),
        StructField("table_id",     StringType(), True),
        StructField("created_at",   StringType(), True),
        StructField("last_altered", StringType(), True),
        StructField("extracted_at", StringType(), False),
        StructField("extraction_method", StringType(), True),
        StructField("detail_error", StringType(), True),
    ])

    table_rows = []
    for r in table_ddl_data:
        row = (
            r['catalog_name'], r['schema_name'], r['table_name'], r['table_type'],
            r.get('storage_format'), r.get('storage_location'), r.get('external_location'),
            r['partition_strategy'], r.get('partition_columns') or [],
            r['clustering_strategy'], r.get('clustering_columns') or [],
            bool(r.get('auto_optimize_write', False)), bool(r.get('auto_optimize_compact', False)),
            r.get('vacuum_retention_hours'), r.get('stats_column_limit'), r.get('stats_custom_columns') or [],
            bool(r.get('cdf_enabled', False)), r.get('time_travel_retention_days'),
            r.get('num_files'), r.get('size_in_bytes'), r.get('size_pretty'), r.get('table_features') or [],
            r.get('table_id'), r.get('created_at'), r.get('last_altered'),
            r['extracted_at'], r.get('extraction_method'), r.get('detail_error')
        )
        table_rows.append(row)

    table_ddl_df = spark.createDataFrame(table_rows, table_schema)
    table_ddl_df.createOrReplaceTempView("table_ddl_info")
    print(f"TABLE_DDL_INFO DataFrame作成完了: {table_ddl_df.count()} レコード")
    display(table_ddl_df.limit(5))

In [ ]:
# COLUMN_DDL_INFO DataFrame作成 (PK/FK対応版)
if column_ddl_data:
    column_schema = StructType([
        StructField("catalog_name",  StringType(), False),
        StructField("schema_name",   StringType(), False),
        StructField("table_name",    StringType(), False),
        StructField("column_name",   StringType(), False),
        StructField("ordinal_position", IntegerType(), False),
        StructField("data_type",     StringType(), True),
        StructField("numeric_precision", IntegerType(), True),
        StructField("numeric_scale",     IntegerType(), True),
        StructField("is_nullable",   BooleanType(), False),
        StructField("default_value", StringType(), True),
        StructField("column_comment", StringType(), True),
        StructField("is_partition_column",  BooleanType(), False),
        StructField("is_clustering_column", BooleanType(), False),
        StructField("is_primary_key",      BooleanType(), False),    # PK情報
        StructField("foreign_key_reference", StringType(), True),   # FK参照情報
        StructField("extracted_at",      StringType(), False),
        StructField("extraction_method", StringType(), True),
    ])

    column_rows = []
    for r in column_ddl_data:
        row = (
            r['catalog_name'], r['schema_name'], r['table_name'], r['column_name'],
            int(r['ordinal_position']), r.get('data_type'), r.get('numeric_precision'), r.get('numeric_scale'),
            bool(r['is_nullable']), r.get('default_value'), r.get('column_comment'),
            bool(r.get('is_partition_column', False)), bool(r.get('is_clustering_column', False)),
            bool(r.get('is_primary_key', False)), r.get('foreign_key_reference'),
            r['extracted_at'], r.get('extraction_method', 'detail_python'),
        )
        column_rows.append(row)

    column_ddl_df = spark.createDataFrame(column_rows, column_schema)
    column_ddl_df.createOrReplaceTempView("column_ddl_info")
    print(f"COLUMN_DDL_INFO DataFrame作成完了: {column_ddl_df.count()} レコード")
    display(column_ddl_df.limit(10))

## 8. 統計情報とPK/FK制約分析

In [ ]:
print("=== メタデータ抽出結果統計 ===")

# 1. テーブル統計
print("\n1. テーブル詳細統計")
spark.sql("""
    SELECT 
      catalog_name, schema_name,
      COUNT(*) AS table_count,
      SUM(CASE WHEN table_type = 'MANAGED' THEN 1 END) AS managed_tables,
      SUM(CASE WHEN table_type = 'EXTERNAL' THEN 1 END) AS external_tables,
      SUM(CASE WHEN table_type IN ('VIEW','MATERIALIZED VIEW') THEN 1 END) AS views,
      SUM(CASE WHEN clustering_strategy != 'NONE' THEN 1 END) AS clustered_tables,
      SUM(CASE WHEN partition_strategy != 'NONE' THEN 1 END) AS partitioned_tables
    FROM table_ddl_info
    GROUP BY catalog_name, schema_name
    ORDER BY catalog_name, schema_name
""").show()

# 2. カラム統計（PK/FK情報含む）
print("\n2. カラム詳細統計（PK/FK含む）")
spark.sql("""
    SELECT
      catalog_name, schema_name,
      COUNT(*) AS total_columns,
      SUM(CASE WHEN is_partition_column THEN 1 END) AS partition_columns,
      SUM(CASE WHEN is_clustering_column THEN 1 END) AS clustering_columns,
      SUM(CASE WHEN is_primary_key THEN 1 END) AS primary_key_columns,
      SUM(CASE WHEN foreign_key_reference IS NOT NULL THEN 1 END) AS foreign_key_columns,
      SUM(CASE WHEN NOT is_nullable THEN 1 END) AS not_null_columns
    FROM column_ddl_info
    GROUP BY catalog_name, schema_name
    ORDER BY catalog_name, schema_name
""").show()

# 3. FK参照関係の詳細
print("\n3. FK参照関係詳細（TOP10）")
spark.sql("""
    SELECT
      concat(catalog_name,'.',schema_name,'.',table_name,'.',column_name) AS source_column,
      foreign_key_reference AS target_reference
    FROM column_ddl_info
    WHERE foreign_key_reference IS NOT NULL
    ORDER BY catalog_name, schema_name, table_name, column_name
    LIMIT 10
""").show(truncate=False)

## 9. メタデータテーブル保存

In [ ]:
# 統合設定を使用したデータ保存とエクスポート
meta_catalog = CONFIG["output_catalog"]
meta_schema  = CONFIG["target_catalog"]

# メタデータ保存先がない場合、作成
spark.sql(f"CREATE CATALOG IF NOT EXISTS `{meta_catalog}`")
spark.sql(f"CREATE SCHEMA IF NOT EXISTS `{meta_catalog}`.`{meta_schema}`")

table_df  = spark.sql("SELECT * FROM table_ddl_info")
column_df = spark.sql("SELECT * FROM column_ddl_info")

# スナップショット時刻を付与
snap_ts = datetime.now(timezone.utc)
table_df  = table_df.withColumn("snapshot_at", F.lit(snap_ts))
column_df = column_df.withColumn("snapshot_at", F.lit(snap_ts))

# 最新版テーブルとして上書き保存（Managed Delta）
(table_df
 .write.mode("overwrite")
 .option("overwriteSchema", "true")
 .saveAsTable(f"`{meta_catalog}`.`{meta_schema}`.`{meta_schema}_table_ddl_info`"))

(column_df
 .write.mode("overwrite")
 .option("overwriteSchema", "true")
 .saveAsTable(f"`{meta_catalog}`.`{meta_schema}`.`{meta_schema}_column_ddl_info`"))

# 保持期間設定
retention_days = CONFIG["retention_days"]

spark.sql(f"""  
        ALTER TABLE `{meta_catalog}`.`{meta_schema}`.`{meta_schema}_table_ddl_info`  SET TBLPROPERTIES
        ('delta.logRetentionDuration'='interval {retention_days} days',
        'delta.deletedFileRetentionDuration'='interval {retention_days} days')
        """)

spark.sql(f"""
        ALTER TABLE `{meta_catalog}`.`{meta_schema}`.`{meta_schema}_column_ddl_info` SET TBLPROPERTIES
        ('delta.logRetentionDuration'='interval {retention_days} days',
        'delta.deletedFileRetentionDuration'='interval {retention_days} days')
        """)

print(f"メタデータ保存完了: {meta_catalog}.{meta_schema}.*_ddl_info")
print(f"\nUnity Catalog メタデータ抽出が完了しました!")
print(f"TABLE_DDL_INFO: {len(table_ddl_data)} テーブル")
print(f"COLUMN_DDL_INFO: {len(column_ddl_data)} カラム")
print(f"PK/FK制約情報も含まれています")